In [35]:
import numpy as np
import pandas as pd

from sklearn.naive_bayes import CategoricalNB
from sklearn.model_selection import StratifiedKFold
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.naive_bayes import MultinomialNB

import re
import seaborn as sns


In [6]:
df_scams=pd.read_csv('../data/output_scam_profiles.csv')
df_real=pd.read_csv('../data/output_real_profiles.csv')

In [14]:
y = df_combined['scam']
X = df_combined.drop(columns=['scam', 'description', 'age', 'location'])

In [15]:
X

,ethnicity,occupation,status,age_group,country
0,asian,other,separated,31-40,Sweden
1,white,other,single,21-30,Spain
2,white,fashion,single,31-40,United States
3,black,other,single,21-30,United States
4,middle eastern,self-employed,single,51-60,Perú
...,...,...,...,...,...
5531,hispanic,legal,single,31-40,Perú
5532,black,service,single,51-60,USA
5533,white,self-employed,single,31-40,Switzerland
5534,white,manager,divorced,51-60,UK


In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [26]:
ohe = OneHotEncoder(handle_unknown="ignore")
ohe.fit(X_train.loc[:, cols_to_categorical])

OneHotEncoder(handle_unknown='ignore')

In [27]:
X_train = ohe.transform(X_train.loc[:, cols_to_categorical]).toarray()
X_test = ohe.transform(X_test.loc[:, cols_to_categorical]).toarray()

In [32]:
mnb = MultinomialNB()
mnb.fit(X_train, y_train)
y_pred = mnb.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.96      0.95      0.95       432
           1       0.97      0.97      0.97       676

    accuracy                           0.96      1108
   macro avg       0.96      0.96      0.96      1108
weighted avg       0.96      0.96      0.96      1108



## Output probabilities instead

In [33]:
y_pred_proba = mnb.predict_proba(X_test)
y_pred_proba

array([[4.82933666e-04, 9.99517066e-01],
       [2.89320611e-05, 9.99971068e-01],
       [9.97243155e-01, 2.75684459e-03],
       ...,
       [8.31542029e-05, 9.99916846e-01],
       [9.99830663e-01, 1.69336712e-04],
       [6.85515740e-06, 9.99993145e-01]])

In [36]:
# calculate softmax for y_pred_proba
y_pred_proba_softmax = np.exp(y_pred_proba) / np.sum(np.exp(y_pred_proba), axis=1)[:, None]
y_pred_proba_softmax

array([[0.26913136, 0.73086864],
       [0.2689528 , 0.7310472 ],
       [0.72997314, 0.27002686],
       ...,
       [0.26897412, 0.73102588],
       [0.73099199, 0.26900801],
       [0.26894412, 0.73105588]])

In [37]:
# extract just the probability for class = 1
y_pred_proba_class_1 = y_pred_proba_softmax[:, 1]
y_pred_proba_class_1

array([0.73086864, 0.7310472 , 0.27002686, ..., 0.73102588, 0.26900801,
       0.73105588])

In [139]:
# encode remaining variables with one-hot encoding

encoder = ce.OneHotEncoder(cols=['ethnicity', 'occupation', 'status', 'age_group'])

X_train = encoder.fit_transform(X_train)

X_test = encoder.transform(X_test)

In [140]:
cols = X_train.columns

In [141]:
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()

X_train = scaler.fit_transform(X_train)

X_test = scaler.transform(X_test)

In [142]:
X_train = pd.DataFrame(X_train, columns=[cols])

In [143]:
X_test = pd.DataFrame(X_test, columns=[cols])

In [144]:
# instantiate the model
gnb = GaussianNB()


# fit the model
gnb.fit(X_train, y_train)

GaussianNB()

In [145]:
y_pred = gnb.predict(X_test)

y_pred

array([0, 1, 0, ..., 1, 0, 1])

In [150]:
print('Model accuracy score: {0:0.4f}'. format(accuracy_score(y_test, y_pred)))
print('Model precision score: {0:0.4f}'. format(precision_score(y_test, y_pred)))
print('Model recall score: {0:0.4f}'. format(recall_score(y_test, y_pred)))
print('Model f1 score: {0:0.4f}'. format(f1_score(y_test, y_pred)))


Model accuracy score: 0.7951
Model precision score: 0.8812
Model recall score: 0.7678
Model f1 score: 0.8206


In [151]:
y_pred_train = gnb.predict(X_train)

y_pred_train

array([1, 1, 0, ..., 0, 0, 0])

In [152]:
print('Training-set accuracy score: {0:0.4f}'. format(accuracy_score(y_train, y_pred_train)))

Training-set accuracy score: 0.8060


In [164]:
Test=pd.concat([X_train, y_train], axis=1)

In [169]:
Test.corr()
#sns.heatmap(Test.corr())

,"(ethnicity_1,)","(ethnicity_2,)","(ethnicity_3,)","(ethnicity_4,)","(ethnicity_5,)","(ethnicity_6,)","(ethnicity_7,)","(ethnicity_8,)","(ethnicity_9,)","(ethnicity_10,)",...,"(status_6,)","(age_group_1,)","(age_group_2,)","(age_group_3,)","(age_group_4,)","(age_group_5,)","(age_group_6,)","(age_group_7,)","(age_group_8,)",scam
"(ethnicity_1,)",1.000000,-0.298544,-0.308100,-0.198589,-0.541706,-0.280973,-0.141876,-0.055464,-0.068997,-0.055464,...,-0.040996,-0.080135,0.114715,-0.018415,0.041100,-0.065272,0.043077,-0.003182,-0.010078,-0.025440
"(ethnicity_2,)",-0.298544,1.000000,-0.074465,-0.047997,-0.130925,-0.067908,-0.034290,-0.013405,-0.016676,-0.013405,...,-0.018094,0.010733,0.036572,-0.005869,-0.024568,-0.026486,-0.019798,-0.008077,-0.015661,0.008597
"(ethnicity_3,)",-0.308100,-0.074465,1.000000,-0.049533,-0.135116,-0.070082,-0.035388,-0.013834,-0.017210,-0.013834,...,0.020576,0.033668,-0.037316,-0.001404,-0.010504,0.020185,-0.012552,-0.008336,-0.001064,-0.002741
"(ethnicity_4,)",-0.198589,-0.047997,-0.049533,1.000000,-0.087090,-0.045172,-0.022810,-0.008917,-0.011093,-0.008917,...,0.007415,0.010813,-0.043516,0.003754,0.015212,0.023682,-0.006434,0.038041,0.012029,0.034918
"(ethnicity_5,)",-0.541706,-0.130925,-0.135116,-0.087090,1.000000,-0.123219,-0.062219,-0.024323,-0.030258,-0.024323,...,0.052684,0.077406,-0.065004,0.051967,-0.010217,-0.061195,-0.029633,0.004431,-0.028416,-0.011919
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(age_group_5,)",-0.065272,-0.026486,0.020185,0.023682,-0.061195,0.218011,0.019918,-0.019336,0.008465,-0.005870,...,-0.016102,-0.224377,-0.234757,-0.234487,-0.103329,1.000000,-0.040987,-0.011651,-0.022590,-0.009541
"(age_group_6,)",0.043077,-0.019798,-0.012552,-0.006434,-0.029633,-0.008572,0.020870,-0.005279,-0.006567,-0.005279,...,0.025070,-0.061257,-0.064091,-0.064017,-0.028210,-0.040987,1.000000,-0.003181,-0.006167,-0.010897
"(age_group_7,)",-0.003182,-0.008077,-0.008336,0.038041,0.004431,-0.007602,-0.003838,-0.001501,-0.001867,-0.001501,...,-0.002025,-0.017413,-0.018218,-0.018197,-0.008019,-0.011651,-0.003181,1.000000,-0.001753,0.020094
"(age_group_8,)",-0.010078,-0.015661,-0.001064,0.012029,-0.028416,0.067046,0.023501,-0.002909,-0.003619,-0.002909,...,-0.003927,-0.033762,-0.035324,-0.035283,-0.015548,-0.022590,-0.006167,-0.001753,1.000000,-0.019780


In [181]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
high_correlation = Test.corr()[Test.corr() <-0.5]
print(high_correlation)

                  (ethnicity_1,)  (ethnicity_2,)  (ethnicity_3,)  \
(ethnicity_1,)               NaN             NaN             NaN   
(ethnicity_2,)               NaN             NaN             NaN   
(ethnicity_3,)               NaN             NaN             NaN   
(ethnicity_4,)               NaN             NaN             NaN   
(ethnicity_5,)         -0.541706             NaN             NaN   
(ethnicity_6,)               NaN             NaN             NaN   
(ethnicity_7,)               NaN             NaN             NaN   
(ethnicity_8,)               NaN             NaN             NaN   
(ethnicity_9,)               NaN             NaN             NaN   
(ethnicity_10,)              NaN             NaN             NaN   
(ethnicity_11,)              NaN             NaN             NaN   
(ethnicity_12,)              NaN             NaN             NaN   
(ethnicity_13,)              NaN             NaN             NaN   
(occupation_1,)              NaN             NaN